## CIFAR 10

In [99]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [100]:
from fastai.conv_learner import *
from fastai.models.cifar10.wideresnet import wrn_22
torch.backends.cudnn.benchmark = True
PATH = Path("data/cifar10/")
os.makedirs(PATH,exist_ok=True)

In [101]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

bs=512
sz=32
workers=8

In [150]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets

def pad(img, p=4, padding_mode='reflect'):
        return Image.fromarray(np.pad(np.asarray(img), ((p, p), (p, p), (0, 0)), padding_mode))

def to_pil(img):
    print(type(img))
#     print(img)
    return Image.fromarray(img)

def torch_tfms(size, conv_pil=False, to_numpy=False):
    mean,std=[0.4914 , 0.48216, 0.44653], [0.24703, 0.24349, 0.26159]
    normalize = transforms.Normalize(mean=mean, std=std)
    tfms = [transforms.ToTensor(), normalize]
    
    if to_numpy: tfms = [np.array, Normalize(mean,std), lambda x: x[0].T]
        
        
    aug_tfms = [
        pad, # TODO: use `padding` rather than assuming 4
        transforms.RandomCrop(size),
        transforms.ColorJitter(.25,.25,.25),
        transforms.RandomRotation(2),
        transforms.RandomHorizontalFlip(),
    ]
    scale_size = 40
    padding = int((scale_size - size) / 2)
    
    train_tfms = transforms.Compose(aug_tfms + tfms)
    train_tfms.sz = size
    val_tfms = transforms.Compose(tfms)
    val_tfms.sz = size
    if conv_pil:
        train_tfms.transforms.insert(0, to_pil)
        val_tfms.transforms.insert(0, to_pil)
    return train_tfms, val_tfms
    
def torch_ds(data_path, tfms):
    train_tfms, val_tfms = tfms
    
    # Data loading code
    traindir = os.path.join(data_path, 'train')
    valdir = os.path.join(data_path, 'test')

    train_dataset = datasets.ImageFolder(traindir, train_tfms)
    val_dataset = datasets.ImageFolder(valdir, val_tfms)
    return train_dataset, val_dataset

def torch_loader(data_path, datasets):
    train_dataset, val_dataset = datasets
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=bs, shuffle=True,
        num_workers=workers, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=bs*2, shuffle=False,
        num_workers=workers, pin_memory=True)

    data = ModelData(data_path, train_loader, val_loader)
    return data


In [103]:
def get_learner(data):
    m = wrn_22()
    learn = ConvLearner.from_model_data(m, data)
    learn.crit = nn.CrossEntropyLoss()
    learn.metrics = [accuracy]
    return learn

### Torch Data Loader

In [8]:
tfms = torch_tfms(sz)
dsets = torch_ds(PATH, tfms)
torch_data = torch_loader(str(PATH), dsets)

In [9]:
learn = get_learner(torch_data)

In [10]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

 30%|██▉       | 29/98 [00:10<00:23,  2.88it/s, loss=2.54]

Process Process-4:
Process Process-5:
Process Process-6:
Process Process-8:
Process Process-7:
Process Process-3:
Process Process-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Process Process-1:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, 

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run


### Fastai Data Loader

In [11]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fastai_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [12]:
learn = get_learner(fastai_data)

In [13]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

  0%|          | 0/98 [00:00<?, ?it/s]                    


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



 39%|███▉      | 38/98 [00:12<00:19,  3.08it/s, loss=2.37]


KeyboardInterrupt: 

### Fastai Default Collate

In [14]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fastai_dc_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [15]:
# Hack to get the hidden collate function out of the torch dataloader class. 
collate_fn = torch.utils.data.DataLoader(None).collate_fn

In [16]:
fastai_dc_data.trn_dl.collate_fn = collate_fn
fastai_dc_data.val_dl.collate_fn = collate_fn

In [18]:
learn = get_learner(fastai_dc_data)

In [ ]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

100%|██████████| 98/98 [00:30<00:00,  3.25it/s, loss=2.08]

### Fastai DataLoader - Torch Transforms

In [151]:
%pdb off

Automatic pdb calling has been turned OFF


In [152]:
tfms = torch_tfms(sz, to_numpy=True)

In [153]:
fdl_tds_data = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [154]:
fdl_tds_data.trn_ds.open_fn = Image.open
fdl_tds_data.val_ds.open_fn = Image.open

In [155]:
learn = get_learner(fdl_tds_data)

In [156]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

 45%|████▍     | 44/98 [00:34<00:41,  1.29it/s, loss=2.29]

KeyboardInterrupt: 

In [157]:
dt = tfms[0]
f1 = fdl_tds_data.trn_ds.fnames[0]

In [158]:
cv2img = cv2.imread(str(PATH/f1))
pimg = Image.open(str(PATH/f1))

In [159]:
pltfm = to_pil(cv2img)

<class 'numpy.ndarray'>


In [161]:
dt(cv2img).shape

(3, 32, 32)

In [163]:
dt(Image.open(str(PATH/f1))).shape

(3, 32, 32)

### Torch DataLoader - Fastai Transforms

In [169]:
tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomCrop(sz), RandomFlip()], pad=sz//8)
fdl = ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)
tdl_fds = fdl.trn_ds, fdl.val_ds
torch_data = torch_loader(str(PATH), tdl_fds)

In [170]:
learn = get_learner(torch_data)

In [171]:
%time learn.fit(lrs=1, n_cycle=1, wds=1e-4, cycle_len=2)

epoch      trn_loss   val_loss   accuracy                 
    0      2.127508   2.06523    0.2073    
 21%|██▏       | 21/98 [00:05<00:19,  3.85it/s, loss=2.07]

Process Process-26:
Process Process-28:
Process Process-30:
Process Process-25:
Process Process-32:
Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f95f0d28a40>
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm_notebook.py", line 187, in __iter__
    yield obj
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 175, in handler
    _error_if_any_worker_fails()
RuntimeError: DataLoader worker (pid 7763) exited unexpectedly with exit code 1.


KeyboardInterrupt: 

Process Process-31:
Process Process-27:
Process Process-29:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/p